In [6]:
import requests
import os
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup
import pandas as pd

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('lang=ko_KR')
path = 'C:/Users/82108/Desktop/chromedriver.exe'

driver = webdriver.Chrome(path) # chromedriver 열기


def main():
    global driver, load_wb, review_num, searchList,place_name
    
    searchList = []

    driver.implicitly_wait(4)  # 렌더링 될때까지 기다린다 4초
    driver.get('https://map.kakao.com/?')  # 주소 가져오기

    # 검색할 목록
    place_infos = ['신촌 카페']

    for i, place in enumerate(place_infos):
        # delay
        if i % 4 == 0 and i != 0:
            sleep(5)
        print("#####", i)
        search(place)
        
    driver.quit()   
    print(searchList)    
    print("finish")
    

def search(place):
    global driver, place_name

    search_area = driver.find_element_by_xpath(
        '//*[@id="search.keyword.query"]')  # 검색 창
    
    search_area.send_keys(place)  # 검색어 입력
    driver.find_element_by_xpath(
        '//*[@id="search.keyword.query"]').send_keys(Keys.ENTER)  # Enter로 검색
    sleep(1)

    # 검색된 정보가 있는 경우에만 탐색
    # 1번 페이지 place list 읽기
    html = driver.page_source

    soup = BeautifulSoup(html, 'html.parser')
    place_lists = soup.select('.placelist > .PlaceItem')  # 검색된 장소 목록

    # 검색된 첫 페이지 장소 목록 크롤링하기
    crawling(place, place_lists)
    print(place_lists)
    search_area.clear()

    # 우선 더보기 클릭해서 2페이지
    try:
        driver.find_element_by_xpath(
            '//*[@id="info.search.place.more"]').send_keys(Keys.ENTER)
        sleep(1)

        while True:
            # 2~ 5페이지 읽기
            for i in range(2, 6):
                # 페이지 넘기기
                xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
                driver.find_element_by_xpath(xPath).send_keys(Keys.ENTER)
                sleep(1)

                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                place_lists = soup.select('.placelist > .PlaceItem')  # 장소 목록 list

                print(place_lists)

                crawling(place, place_lists)
            
            xPath = '//*[@id="info.search.page.next"]'
            driver.find_element_by_xpath(xPath).send_keys(Keys.ENTER)
            sleep(1)

    except ElementNotInteractableException:
        print('not found')
    finally:
        search_area.clear()

def crawling(place, place_lists):
    """
    페이지 목록을 받아서 크롤링 하는 함수
    :param place: 리뷰 정보 찾을 장소이름
    """
    while_flag = False
    for i, place in enumerate(place_lists):
        # 광고에 따라서 index 조정해야함
        if i >= 3:
            i += 1

        place_name = place.select('.head_item > .tit_name > .link_name')[0].text  # place name
        place_address = place.select('.info_item > .addr > p')[0].text  # place address

        detail_page_xpath = '//*[@id="info.search.place.list"]/li[' + str(i + 1) + ']/div[5]/div[4]/a[1]'
        driver.find_element_by_xpath(detail_page_xpath).send_keys(Keys.ENTER)
        driver.switch_to.window(driver.window_handles[-1])  # 상세정보 탭으로 변환
        sleep(1)

        print('####', place_name)

        # 첫 페이지
        extract_review(place_name)

        # 2-5 페이지
        idx = 3
        try:
            page_num = len(driver.find_elements_by_class_name('link_page')) # 페이지 수 찾기
            for i in range(page_num-1):
                # css selector를 이용해 페이지 버튼 누르기
                driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                sleep(1)
                extract_review(place_name)
                idx += 1
            driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 5페이지가 넘는 경우 다음 버튼 누르기
            sleep(1)
            extract_review(place_name) # 리뷰 추출
        except (NoSuchElementException, ElementNotInteractableException):
            print("no review in crawling")

        # 그 이후 페이지
        while True:
            idx = 4
            try:
                page_num = len(driver.find_elements_by_class_name('link_page'))
                for i in range(page_num-1):
                    driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                    sleep(1)
                    extract_review(place_name)
                    idx += 1
                driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 10페이지 이상으로 넘어가기 위한 다음 버튼 클릭
                sleep(1)
                extract_review(place_name) # 리뷰 추출
            except (NoSuchElementException, ElementNotInteractableException):
                print("no review in crawling")
                break

        driver.close()
        driver.switch_to.window(driver.window_handles[0])  # 검색 탭으로 전환

def extract_review(place_name):
    global driver

    ret = True

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 첫 페이지 리뷰 목록 찾기
    review_lists = soup.select('.list_evaluation > li')   
    
    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        
        for i, review in enumerate(review_lists):
            
            comment = review.select('.txt_comment > span') # 리뷰
            rating = review.select('.grade_star > em') # 별점
            val = ''
            if len(comment) != 0:
                innersearchList = []
                if len(rating) != 0:
                    val = comment[0].text + '/' + rating[0].text.replace('점', '')
                    innersearchList.append(place_name)    
                    innersearchList.append(comment[0].text)
                    innersearchList.append(rating[0].text)
                else:
                    val = comment[0].text + '/0'
                    innersearchList.append(place_name)    
                    innersearchList.append(comment[0].text)
                    innersearchList.append(0)
                print(val)
                
                searchList.append(innersearchList)
    else:
        print('no review in extract')
        ret = False
    return ret
    
if __name__ == "__main__":
    main()

##### 0
#### 더파이홀
연대 다닐 때 친구들 놀러오면 무조건 데려가는 곳이었음.(지금 위치 아니고 굴다리 옆에 있었을 당시) 오랜만에 와서 먹어보니 그때처럼 최고다 싶은 정도는 아니지만 얼그레이가나슈파이는 여전히 감동적...!/4
맛있어용/4
보통이에요/3
신세계, 그러나 사악한 가격 ㅜㅜ 그리고 화장실 가면 화장실 나방? 맨날 있음....../3
분위기도 좋고 맛있었습니다 파이 먹고싶을때 첫본째로 생각나는 것이에요  /5
맛있음.친절 피넛초코ㅡ맛있음, 바삭 다음에 딴것도 먹어볼것. 재방문의사ㅡ85퍼/5
초코파이.. 찐한 맛 ㅠ/5
전 그냥 그랬어요 ...ㅎㅎ/3
위생 관리 제발 하세요 코로나가 아니어도 기본적으로 음식 위에 뭘 덮어놔야 하는거 아니에요?/1
예전부터 자주 갔어서 오늘도 포장하러 갔었는데, 위생에 정 떨어졌어요ㅠㅠ  로투스 타르트 주문해놓고 스콘도 시킬까 싶어서 봤는데, 스콘 들어있던 진열장? 안에 큰 날파리가 3-4마리 있었어요. 당연히 스콘 위에도 앉았고요.  좀 아닌 거 같아서 말씀드렸는데, 그 스콘들을 따로 빼는 게 아니라 그냥 맨손으로 (스콘에 맨손 닿음) 날파리 쫓으셨어요. 별 신경 .../1
일부 메뉴들이 진열장 밖에 있어서 조금 위생적인 부분이 신경쓰이기는 하는데 파이는 맛있당 오레오말차가나슈 최고,,,, 항상 사람들이 붐벼서 요즘은 잘 못가겠지만 파이 존맛/4
최고예요/5
맛있었어요~ > </4
보통이에요/3
얼그레이 맛있어서 눈물 한방울/5
밀크티가 너무 써요.... 따로 준 시럽 다 넣었는데도 쓴 맛이 나더라구요../4
언제나 만족. 사람 밀도가 높은 것은 감내해야./5
맛은 4점인데   분위기가 최악.... 정말 최악.... 시장통 + 구린 인테리어/3
좋아요/4
맛있다 인테리어가 먼지쌓이고 좀 정신사납지만 어차피 파이먹으러 가는 곳이니 괜찮다/5
최고예요/5
신촌에 친구들 놀러오면 무조건 데려가는 카페! 분위기도 나름 좋고 얼그레이 가나슈 존맛.. 파이 종류 왠만하면 다 맛있어요 음료는 그냥저냥입니당/5
이제 화

WebDriverException: Message: chrome not reachable
  (Session info: chrome=88.0.4324.182)


In [ ]:
searchList

In [45]:
df = pd.DataFrame(searchList)

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1361 entries, 0 to 1360
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       1361 non-null   object
 1   1       1361 non-null   object
 2   2       1361 non-null   object
dtypes: object(3)
memory usage: 32.0+ KB


In [47]:
df.head(10)

,0,1,2
0,맥심플랜트,분위기도 좋고 커피도 맛있고 공간이 머쉿다~,4점
1,맥심플랜트,최고예요,5점
2,맥심플랜트,좋아요,4점
3,맥심플랜트,공부하기 좋고 넓어서 애용하는 곳,5점
4,맥심플랜트,좋아요,4점
5,맥심플랜트,이가격에 이공간 이맛 끄덕끄덕,5점
6,맥심플랜트,최고예요,5점
7,맥심플랜트,음료보다 인테리어가 더 인상깊어요 넓고 공간도 다양하게 있습니다 꼭대기층 테라스자리...,4점
8,맥심플랜트,요 근처 회사 다니는 사람이에요. 커피 맛은 대단히 맛있지도 별로지도 아닌 보통의 ...,3점
9,맥심플랜트,커피맛 최고! 아메리카노 골든스카이 + 글래머러스 펭귄 케익 최고!! 공간 다양 ...,4점


In [48]:
df.to_csv("왕십리 cafe_review.csv")

In [19]:
cafe_namelist = cafe_list.drop_duplicates()
cafe_namelist

NameError: name 'cafe_list' is not defined

In [ ]:
cafe_namelist = pd.DataFrame(cafe_namelist)
cafe_namelist

In [ ]:
cafe_namelist['cafe_name'] = "강남카페 " + cafe_namelist[0]

In [ ]:
cafe_namelist

In [ ]:
cafe_namelist.drop([0], axis=1)

In [ ]:
cafe_namelist.to_csv("cafe_namelist.csv")